In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import torch
import os, sys
import re, string
from collections import Counter
import csv

In [2]:
!pip install verstack

  Created wheel for verstack: filename=verstack-0.3.1-cp36-none-any.whl size=14343 sha256=93d7179b9b56d9410424db5c410cccf4ff92d6ed6dac496aa4beaa2844926d7c
  Stored in directory: /root/.cache/pip/wheels/15/1b/58/10e59516150cea4d9b1dbacceb3bffcc0cfd2d166efabec2f6
Successfully built verstack


In [3]:
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import mean_absolute_error
from verstack.stratified_continuous_split import scsplit 

In [4]:
!pip install simpletransformers

     |████████████████████████████████| 225kB 8.8MB/s 
     |████████████████████████████████| 2.9MB 12.6MB/s 
     |████████████████████████████████| 71kB 11.1MB/s 
     |████████████████████████████████| 7.4MB 31.5MB/s 
     |████████████████████████████████| 1.3MB 52.5MB/s 
     |████████████████████████████████| 1.8MB 54.5MB/s 
     |████████████████████████████████| 317kB 55.4MB/s 
     |████████████████████████████████| 51kB 8.3MB/s 
     |████████████████████████████████| 102kB 13.7MB/s 
     |████████████████████████████████| 4.5MB 48.9MB/s 
     |████████████████████████████████| 133kB 56.2MB/s 
     |████████████████████████████████| 112kB 58.5MB/s 
     |████████████████████████████████| 6.9MB 50.2MB/s 
     |████████████████████████████████| 163kB 58.2MB/s 
     |████████████████████████████████| 890kB 53.0MB/s 
     |████████████████████████████████| 1.1MB 48.7MB/s 
     |████████████████████████████████| 133kB 51.6MB/s 
     |████████████████████████████████| 102kB 14.7MB

In [5]:
# Importing the libraries needed
import transformers
from torch.utils.data import Dataset, DataLoader
from transformers import DistilBertModel, DistilBertTokenizer

In [6]:
from simpletransformers.classification import ClassificationModel, ClassificationArgs
import logging

In [7]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [8]:
# Setting up the device for GPU usage

from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'
device

'cuda'

In [9]:
path = "/content/drive/My Drive/"
path_data_train = path + "/covidChallenge/data/train.csv"

In [10]:
train = pd.read_csv(path_data_train)

In [11]:
train.columns

Index(['id', 'timestamp', 'retweet_count', 'user_verified',
       'user_statuses_count', 'user_followers_count', 'user_friends_count',
       'user_mentions', 'urls', 'hashtags', 'text'],
      dtype='object')

In [12]:
counter_user_mentions = train['user_mentions'].isnull().sum()
counter_urls = train['urls'].isnull().sum()
counter_hashtags = train['hashtags'].isnull().sum()    

In [13]:
print("\t User_mentions column NULL entries: ", counter_user_mentions)
print("\t Urls column column NULL entries: ", counter_urls)
print("\t Hashtags column NULL entries: ", counter_hashtags)
print("\t Number of row: ", len(train.index))

	 User_mentions column NULL entries:  611486
	 Urls column column NULL entries:  451697
	 Hashtags column NULL entries:  593326
	 Number of row:  665777


**Comment:**

- > As noted in the previous cell, we can drop the columns since most of the entries does not provide much information

In [14]:
train.drop('user_mentions', axis=1, inplace=True)
train.drop('urls', axis=1, inplace=True)
train.drop('hashtags', axis=1, inplace=True)

In [15]:
train[:10]

,id,timestamp,retweet_count,user_verified,user_statuses_count,user_followers_count,user_friends_count,text
0,0,1588696955143,0,False,68460,1101,1226,Smh I give up
1,1,1588464948124,0,False,309,51,202,"Most of us are Human Beings, but I think you m..."
2,2,1588634673360,0,False,3241,1675,2325,"Old dirty tricks Trump, at it again...like we ..."
3,3,1588433158672,0,False,32327,667,304,Seriously..... I worked 86 hours my last check...
4,4,1588582751599,0,False,581,42,127,May ALMIGHTY ALLAH have mercy on us all. Only ...
5,5,1588434563287,0,False,7214,503,1126,They couldn’t care less.
6,6,1588692966869,2,False,372,738,472,Extremely valid points being made here 👇🏾 #Eth...
7,7,1588316892450,1,False,2085,3808,153,COVID-19 dominated the discussion Tuesday at a...
8,8,1588625905286,0,False,17765,11666,40,BC now has 112 patients on ventilators. 17 of...
9,9,1588604315931,0,False,3086,66,241,a COVID-19 vaccine would be pretty lit ngl


In [16]:
!pip install emoji

     |████████████████████████████████| 51kB 4.3MB/s 
  Created wheel for emoji: filename=emoji-0.6.0-cp36-none-any.whl size=49716 sha256=d9c865a7474163aa383194d281efba459d730579695fe8d392c8c8ba35f4cef6
  Stored in directory: /root/.cache/pip/wheels/46/2c/8b/9dcf5216ca68e14e0320e283692dce8ae321cdc01e73e17796
Successfully built emoji


In [17]:
import emoji
# Some basic helper functions to clean text by removing urls, emojis, html tags and punctuations.

def remove_URL(text):
    url = re.compile(r'https?://\S+|www\.\S+')
    return url.sub(r'', text)


def remove_emoji(text):
    """
    emoji_pattern = re.compile(
        '['
        u'\U0001F600-\U0001F64F'  # emoticons
        u'\U0001F300-\U0001F5FF'  # symbols & pictographs
        u'\U0001F680-\U0001F6FF'  # transport & map symbols
        u'\U0001F1E0-\U0001F1FF'  # flags (iOS)
        u'\U00002702-\U000027B0'
        u'\U000024C2-\U0001F251'
        ']+',
        flags=re.UNICODE)
    return emoji_pattern.sub(r'', text)
    """
    #return re.sub('[^A-Za-z0-9]+', ' ', text)
    new_text = re.sub(emoji.get_emoji_regexp(), r"", text)
    return new_text


def remove_html(text):
    html = re.compile(r'<.*?>|&([a-z0-9]+|#[0-9]{1,6}|#x[0-9a-f]{1,6});')
    return re.sub(html, '', text)


def remove_punct(text):
    table = str.maketrans('', '', string.punctuation)
    return text.translate(table)

In [18]:
# Applying helper functions
train_clean = train.copy()
train_clean['text'] = train['text'].apply(lambda x: remove_URL(x))
train_clean['text'] = train['text'].apply(lambda x: remove_emoji(x))
train_clean['text'] = train['text'].apply(lambda x: remove_html(x))
train_clean['text'] = train['text'].apply(lambda x: remove_punct(x))

In [19]:
train_clean.head()

,id,timestamp,retweet_count,user_verified,user_statuses_count,user_followers_count,user_friends_count,text
0,0,1588696955143,0,False,68460,1101,1226,Smh I give up
1,1,1588464948124,0,False,309,51,202,Most of us are Human Beings but I think you mi...
2,2,1588634673360,0,False,3241,1675,2325,Old dirty tricks Trump at it againlike we dont...
3,3,1588433158672,0,False,32327,667,304,Seriously I worked 86 hours my last check and ...
4,4,1588582751599,0,False,581,42,127,May ALMIGHTY ALLAH have mercy on us all Only l...


In [20]:
# Defining some key variables that will be used later on in the training
MAX_LEN = 50
TRAIN_BATCH_SIZE = 4
VALID_BATCH_SIZE = 2
EPOCHS = 1
LEARNING_RATE = 1e-05
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-cased')

In [21]:
class Triage(Dataset):
    def __init__(self, dataframe, tokenizer, max_len):
        self.len = len(dataframe)
        self.data = dataframe
        self.tokenizer = tokenizer
        self.max_len = max_len
        
    def __getitem__(self, index):
        title = str(self.data.text[index])
        title = " ".join(title.split())
        inputs = self.tokenizer.encode_plus(
            title,
            None,
            add_special_tokens=True,
            max_length=self.max_len,
            pad_to_max_length=True,
            return_token_type_ids=True,
            truncation=True
        )
        ids = inputs['input_ids']
        #print(ids)
        mask = inputs['attention_mask']

        return {
            'ids': torch.tensor(ids, dtype=torch.long),
            'mask': torch.tensor(mask, dtype=torch.long),
            'targets': torch.tensor(self.data.retweet_count[index], dtype=torch.long)
        } 
    
    def __len__(self):
        return self.len

In [22]:
train_clean = train_clean[['text','retweet_count']]

In [23]:
train_clean[:15]

,text,retweet_count
0,Smh I give up,0
1,Most of us are Human Beings but I think you mi...,0
2,Old dirty tricks Trump at it againlike we dont...,0
3,Seriously I worked 86 hours my last check and ...,0
4,May ALMIGHTY ALLAH have mercy on us all Only l...,0
5,They couldn’t care less,0
6,Extremely valid points being made here 👇🏾 Ethi...,2
7,COVID19 dominated the discussion Tuesday at a ...,1
8,BC now has 112 patients on ventilators 17 of ...,0
9,a COVID19 vaccine would be pretty lit ngl,0


In [24]:
train_clean.retweet_count

0         0
1         0
2         0
3         0
4         0
         ..
665772    0
665773    1
665774    8
665775    0
665776    0
Name: retweet_count, Length: 665777, dtype: int64

In [25]:
# Creating the dataset and dataloader for the neural network

train_size = 0.8
train_dataset= train_clean.sample(frac=train_size,random_state=200)
test_dataset= train_clean.drop(train_dataset.index).reset_index(drop=True)
train_dataset = train_dataset.reset_index(drop=True)

print("FULL Dataset: {}".format(train_clean.shape))
print("TRAIN Dataset: {}".format(train_dataset.shape))
print("TEST Dataset: {}".format(test_dataset.shape))

training_set = Triage(train_dataset, tokenizer, MAX_LEN)
testing_set = Triage(test_dataset, tokenizer, MAX_LEN)

FULL Dataset: (665777, 2)
TRAIN Dataset: (532622, 2)
TEST Dataset: (133155, 2)


In [26]:
train_dataset.retweet_count

0          0
1          0
2          0
3          0
4          0
          ..
532617    89
532618     0
532619     0
532620     0
532621     0
Name: retweet_count, Length: 532622, dtype: int64

In [27]:
training_set.__getitem__(532617)

{'ids': tensor([  101,   151,  1179,  3438,  1186,  1137,  1161,  2617, 12702,   191,
         11848,  1513,   181,  4060, 13601,  2605,  1204,  9183,   185, 19123,
          4267,   187,  8009,  1116,  6052,  1874,   178, 28117,  8136,   191,
          7409, 27909, 28117,  1233,  1884, 15789, 27608,   254,  3073, 13335,
         18637, 10024,  1679, 14596,  1679, 10658,  2858,  4267,  1856,   102]),
 'mask': tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1]),
 'targets': tensor(89)}

In [28]:
train_params = {'batch_size': TRAIN_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 0
                }

test_params = {'batch_size': VALID_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 0
                }

training_loader = DataLoader(training_set, **train_params)
testing_loader = DataLoader(testing_set, **test_params)

In [29]:
# Creating the customized model, by adding a drop out and a dense layer on top of distil bert to get the final output for the model. 

class DistillBERTClass(torch.nn.Module):
    def __init__(self):
        super(DistillBERTClass, self).__init__()
        self.l1 = DistilBertModel.from_pretrained("distilbert-base-uncased")
        self.pre_classifier = torch.nn.Linear(768, 768)
        self.dropout = torch.nn.Dropout(0.3)
        self.regressor = torch.nn.Linear(768, 1)

    def forward(self, input_ids, attention_mask):
        output_1 = self.l1(input_ids=input_ids, attention_mask=attention_mask)
        hidden_state = output_1[0]
        pooler = hidden_state[:, 0]
        pooler = self.pre_classifier(pooler)
        pooler = torch.nn.ReLU()(pooler)
        pooler = self.dropout(pooler)
        output = self.regressor(pooler)
        return output

In [63]:
model = DistillBERTClass()
model.to(device)

DistillBERTClass(
  (l1): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0): TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
            (lin1): Linear(in_feat

In [64]:
path_data_eval = path + "/covidChallenge/data/evaluation.csv"
evaluation = pd.read_csv(path_data_eval)

In [65]:
evaluation.shape

(285334, 10)

In [66]:
# Saving the files for re-use

output_model_file = path + 'covidChallenge/pytorch_distilbert_news.bin'
output_vocab_file = path + 'covidChallenge/vocab_distilbert_news.bin'

In [67]:
#model.load_state_dict(torch.load(output_model_file))
#model = torch.load(output_model_file)
#model.eval()

In [68]:
# Creating the loss function and optimizer
loss_function = torch.nn.MSELoss()
optimizer = torch.optim.Adam(params =  model.parameters(), lr=LEARNING_RATE)

In [69]:
# Function to calcuate the accuracy of the model

def calcuate_accu(predicted, targets):
    n_correct = (predicted==targets).sum().item()
    return n_correct

In [70]:
# Defining the training function on the 80% of the dataset for tuning the distilbert model

def train(epoch):
    tr_loss = 0
    n_correct = 0
    nb_tr_steps = 0
    nb_tr_examples = 0
    model.train()
    for _, data in enumerate(training_loader, 0):
        ids = data['ids'].to(device, dtype = torch.long)
        mask = data['mask'].to(device, dtype = torch.long)
        targets = data['targets'].to(device, dtype = torch.long)

        outputs = model(ids, mask)
        #outputs = torch.tensor(outputs, dtype=torch.cuda.long)
        #print("Outputs", outputs.shape, ", Type:", type(outputs))
        #print("targets", targets.shape, ", Type:", type(targets.T))
        loss = loss_function(outputs[:,0].float(), targets.float())
        tr_loss += loss.item()
        #big_val, big_idx = torch.max(outputs.data, dim=1)
        n_correct += calcuate_accu(outputs, targets)

        nb_tr_steps += 1
        nb_tr_examples+=targets.size(0)
        
        if _%5000==0:
            loss_step = tr_loss/nb_tr_steps
            accu_step = (n_correct*100)/nb_tr_examples 
            print(f"Training Loss per 5000 steps: {loss_step}")
            print(f"Training Accuracy per 5000 steps: {accu_step}")
            print(f"Loss: ", {loss})
            targets_cpu, outputs_cpu = targets.cpu().detach().numpy(), outputs.cpu().detach().numpy()
            print("MAE Prediction error: ", mean_absolute_error(y_true=targets_cpu, y_pred=outputs_cpu))

        optimizer.zero_grad()
        loss.backward()
        # # When using GPU
        optimizer.step()

    print(f'The Total Accuracy for Epoch {epoch}: {(n_correct*100)/nb_tr_examples}')
    epoch_loss = tr_loss/nb_tr_steps
    epoch_accu = (n_correct*100)/nb_tr_examples
    print(f"Training Loss Epoch: {epoch_loss}")
    print(f"Training Accuracy Epoch: {epoch_accu}")

    return 

In [71]:
for epoch in range(EPOCHS):
  train(epoch)

Training Loss per 5000 steps: 1.0547682046890259
Training Accuracy per 5000 steps: 0.0
Loss:  {tensor(1.0548, device='cuda:0', grad_fn=<MseLossBackward>)}
MAE Prediction error:  0.5853186119347811
Training Loss per 5000 steps: 9587202.90926779
Training Accuracy per 5000 steps: 0.0
Loss:  {tensor(390.0812, device='cuda:0', grad_fn=<MseLossBackward>)}
MAE Prediction error:  19.721953868865967
Training Loss per 5000 steps: 7876849.274962111
Training Accuracy per 5000 steps: 0.0
Loss:  {tensor(473.9568, device='cuda:0', grad_fn=<MseLossBackward>)}
MAE Prediction error:  21.74516487121582
Training Loss per 5000 steps: 7222589.458945485
Training Accuracy per 5000 steps: 0.0
Loss:  {tensor(6276.9756, device='cuda:0', grad_fn=<MseLossBackward>)}
MAE Prediction error:  76.76819229125977
Training Loss per 5000 steps: 6974412.568233626
Training Accuracy per 5000 steps: 0.0
Loss:  {tensor(5417.0684, device='cuda:0', grad_fn=<MseLossBackward>)}
MAE Prediction error:  67.60697746276855
Training Loss

In [72]:
def valid(model, testing_loader):
    model.eval()
    tr_loss = 0
    nb_tr_steps = 0
    nb_tr_examples = 0
    n_correct = 0; n_wrong = 0; total = 0
    with torch.no_grad():
        for _, data in enumerate(testing_loader, 0):
            ids = data['ids'].to(device, dtype = torch.long)
            mask = data['mask'].to(device, dtype = torch.long)
            targets = data['targets'].to(device, dtype = torch.long)
            outputs = model(ids, mask).squeeze()
            loss = loss_function(outputs.float(), targets.float())
            
            tr_loss += loss.item()
            #big_val, big_idx = torch.max(outputs.data, dim=1)
            n_correct += calcuate_accu(outputs, targets)

            nb_tr_steps += 1
            nb_tr_examples+=targets.size(0)
            
            if _%5000==0:
                loss_step = tr_loss/nb_tr_steps
                accu_step = (n_correct*100)/nb_tr_examples
                print(f"Validation Loss per 100 steps: {loss_step}")
                print(f"Validation Accuracy per 100 steps: {accu_step}")
                print("Loss: ", loss)
                targets_cpu, outputs_cpu = targets.cpu().detach().numpy(), outputs.cpu().detach().numpy()
                print("MAE Prediction error: ", mean_absolute_error(y_true=targets_cpu, y_pred=outputs_cpu))
    epoch_loss = tr_loss/nb_tr_steps
    epoch_accu = (n_correct*100)/nb_tr_examples
    print(f"Validation Loss Epoch: {epoch_loss}")
    print(f"Validation Accuracy Epoch: {epoch_accu}")
    
    #return epoch_accu
    return epoch_accu

In [80]:
acc = valid(model, testing_loader)
print("Accuracy on test data = %0.2f%%" % acc)

Validation Loss per 100 steps: 9254.181640625
Validation Accuracy per 100 steps: 0.0
Loss:  tensor(9254.1816, device='cuda:0')
MAE Prediction error:  96.19532012939453
Validation Loss per 100 steps: 3873819.4040924967
Validation Accuracy per 100 steps: 0.0
Loss:  tensor(9766.9336, device='cuda:0')
MAE Prediction error:  98.81705093383789
Validation Loss per 100 steps: 4691386.028372553
Validation Accuracy per 100 steps: 0.0
Loss:  tensor(97621296., device='cuda:0')
MAE Prediction error:  7037.107822418213
Validation Loss per 100 steps: 4669274.723016366
Validation Accuracy per 100 steps: 0.0
Loss:  tensor(18360.3379, device='cuda:0')
MAE Prediction error:  131.06295776367188
Validation Loss per 100 steps: 4209244.514656357
Validation Accuracy per 100 steps: 0.0
Loss:  tensor(16557.1758, device='cuda:0')
MAE Prediction error:  125.63214111328125
Validation Loss per 100 steps: 3766731.070879658
Validation Accuracy per 100 steps: 0.0
Loss:  tensor(9921.0742, device='cuda:0')
MAE Predictio

In [81]:
model_to_save = model
torch.save(model_to_save, output_model_file)
tokenizer.save_vocabulary(output_vocab_file)

print('All files saved')
print('This tutorial is completed')

All files saved
This tutorial is completed


In [82]:
EVAL_BATCH_SIZE = 1
eval_params = {'batch_size': EVAL_BATCH_SIZE,
                'shuffle': False,
                'num_workers': 0
                }

In [83]:
class TriageEval(Dataset):
    def __init__(self, dataframe, tokenizer, max_len):
        self.len = len(dataframe)
        self.data = dataframe
        self.tokenizer = tokenizer
        self.max_len = max_len
        
    def __getitem__(self, index):
        title = str(self.data.text[index])
        title = " ".join(title.split())
        inputs = self.tokenizer.encode_plus(
            title,
            None,
            add_special_tokens=True,
            max_length=self.max_len,
            pad_to_max_length=True,
            return_token_type_ids=True,
            truncation=True
        )
        ids = inputs['input_ids']
        #print(ids)
        mask = inputs['attention_mask']

        return {
            'id': torch.tensor(self.data.id[index], dtype=torch.long),
            'ids': torch.tensor(ids, dtype=torch.long),
            'mask': torch.tensor(mask, dtype=torch.long),
            #'targets': torch.tensor(self.data.retweet_count[index], dtype=torch.long)
        } 
    
    def __len__(self):
        return self.len

In [77]:
evaluation_set = TriageEval(evaluation, tokenizer, MAX_LEN)
evaluationloader = DataLoader(evaluation_set, **eval_params)

In [78]:
evaluation_set.__getitem__(14)

{'id': tensor(665790),
 'ids': tensor([  101,  1103,  4157,  1544,   118,  4275,  1147, 22572, 22962,  9828,
          6090,   102,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0]),
 'mask': tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0])}

In [79]:
# And then we predict the values for our testing set
with torch.no_grad():
  f = open("predictions.txt","w+")
  with open("predictions.txt", 'w') as f:
    writer = csv.writer(f)
    writer.writerow(["TweetID", "NoRetweets"])

    for _, data in enumerate(evaluationloader, 0):
        ids = data['ids'].to(device, dtype = torch.long)
        mask = data['mask'].to(device, dtype = torch.long)
        #targets = data['targets'].to(device, dtype = torch.long)
        #print("targets: ", targets)
        output = model(ids, mask).squeeze()
        id = data['id']
        print("Id: {}, prediction: {}".format(id, int(output)))
        writer.writerow([str(id) , str(int(output))])


#y_pred = model(eval)
#print("Prediction error:", mean_absolute_error(y_true=y_test, y_pred=y_pred))

Le flux de sortie a été tronqué et ne contient que les 5000 dernières lignes.
Id: tensor([946110]), prediction: 167
Id: tensor([946111]), prediction: 167
Id: tensor([946112]), prediction: 95
Id: tensor([946113]), prediction: 167
Id: tensor([946114]), prediction: 167
Id: tensor([946115]), prediction: 167
Id: tensor([946116]), prediction: 101
Id: tensor([946117]), prediction: 100
Id: tensor([946118]), prediction: 167
Id: tensor([946119]), prediction: 100
Id: tensor([946120]), prediction: 167
Id: tensor([946121]), prediction: 167
Id: tensor([946122]), prediction: 167
Id: tensor([946123]), prediction: 101
Id: tensor([946124]), prediction: 97
Id: tensor([946125]), prediction: 102
Id: tensor([946126]), prediction: 96
Id: tensor([946127]), prediction: 100
Id: tensor([946128]), prediction: 167
Id: tensor([946129]), prediction: 167
Id: tensor([946130]), prediction: 95
Id: tensor([946131]), prediction: 167
Id: tensor([946132]), prediction: 167
Id: tensor([946133]), prediction: 167
Id: tensor([94